# ADA Project
## Modeling

## 1. ARIMA

##### Check for stationnarity

In [ ]:
from statsmodels.tsa.stattools import adfuller

# Perform ADF test on a given time series
def perform_adf_test(series):
    result = adfuller(series, autolag='AIC')  
    print(f'ADF Statistic: {result[0]}')
    print(f'p-value: {result[1]}')
    print('Critical Values:')
    for key, value in result[4].items():
        print(f'\t{key}: {value}')
    
for wolf_id, group in data.groupby('individual-id'):
    print(f'Performing ADF Test on Wolf {wolf_id}')
    print('Latitude:')
    perform_adf_test(group['location-lat'])
    print('\nLongitude:')
    perform_adf_test(group['location-long'])
    print('\n')


##### ARIMA model

In [ ]:
import pandas as pd
from pmdarima import auto_arima
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np


# Load and preprocess the data
data = pd.read_csv('../Data/merged_data.csv')
data['timestamp'] = pd.to_datetime(data['timestamp'])
data.set_index('timestamp', inplace=True)

# Filter for 'Wolf'
data = data[data['animal-type'] == 'Wolf']

# Initialize results dictionary
results = {}

# Loop over each wolf's data
for wolf_id, group in data.groupby('individual-id'):
    split_index = int(len(group) * 0.8)
    train_data = group.iloc[:split_index]
    test_data = group.iloc[split_index:]

    # Fit ARIMA model on the training data for latitude and longitude
    model_lat = auto_arima(train_data['location-lat'], seasonal=False, stepwise=True, max_p=2, max_q=2, d=0, suppress_warnings=True)
    model_long = auto_arima(train_data['location-long'], seasonal=False, stepwise=True, max_p=2, max_q=2, d=0, suppress_warnings=True)
    
    # Predict on the training set to assess training accuracy
    train_preds_lat = model_lat.predict(n_periods=len(train_data))
    train_preds_long = model_long.predict(n_periods=len(train_data))

    # Predict on the test set
    test_preds_lat = model_lat.predict(n_periods=len(test_data))
    test_preds_long = model_long.predict(n_periods=len(test_data))

    # Compute accuracy metrics for training
    train_mae_lat = mean_absolute_error(train_data['location-lat'], train_preds_lat)
    train_rmse_lat = np.sqrt(mean_squared_error(train_data['location-lat'], train_preds_lat))
    train_mae_long = mean_absolute_error(train_data['location-long'], train_preds_long)
    train_rmse_long = np.sqrt(mean_squared_error(train_data['location-long'], train_preds_long))

    # Compute accuracy metrics for testing
    test_mae_lat = mean_absolute_error(test_data['location-lat'], test_preds_lat)
    test_rmse_lat = np.sqrt(mean_squared_error(test_data['location-lat'], test_preds_lat))
    test_mae_long = mean_absolute_error(test_data['location-long'], test_preds_long)
    test_rmse_long = np.sqrt(mean_squared_error(test_data['location-long'], test_preds_long))

    # Store results
    results[wolf_id] = {
        'training_accuracy': {'MAE_lat': train_mae_lat, 'RMSE_lat': train_rmse_lat, 'MAE_long': train_mae_long, 'RMSE_long': train_rmse_long},
        'test_accuracy': {'MAE_lat': test_mae_lat, 'RMSE_lat': test_rmse_lat, 'MAE_long': test_mae_long, 'RMSE_long': test_rmse_long},
        'test_predictions': (test_preds_lat, test_preds_long),
        'actual_test_data': (test_data['location-lat'].values, test_data['location-long'].values)
    }

# Display results for each wolf
for wolf_id, info in results.items():
    print(f"Wolf ID: {wolf_id}")
    print("Training Accuracy:", info['training_accuracy'])
    print("Test Accuracy:", info['test_accuracy'])

#### Map of the predictions

In [ ]:
import folium

# Function to create a map for a single wolf
def create_wolf_map(wolf_data, test_preds_lat, test_preds_long, wolf_id):
    # Starting point for the map
    start_lat = wolf_data['location-lat'].iloc[0]
    start_long = wolf_data['location-long'].iloc[0]
    wolf_map = folium.Map(location=[start_lat, start_long], zoom_start=8)

    # Split the data into training and testing
    split_index = int(len(wolf_data) * 0.8)
    train_data = wolf_data.iloc[:split_index]
    test_data = wolf_data.iloc[split_index:]

    # Define the HTML code for a green cross
    html_green_cross = '''
    <div style="position: relative; width: 8px; height: 8px;">
        <div style="position: absolute; top: 50%; left: 0; transform: translate(0%, -50%); width: 100%; height: 2px; background-color: purple;"></div>
        <div style="position: absolute; top: 0; left: 50%; transform: translate(-50%, 0%); width: 2px; height: 100%; background-color: purple;"></div>
    </div>
    '''
    
    # Add training data points and draw lines between them
    train_points = [(row['location-lat'], row['location-long']) for idx, row in train_data.iterrows()]
    folium.PolyLine(train_points, color="blue", weight=2.5, opacity=1).add_to(wolf_map)
    for point in train_points:
        folium.CircleMarker(
            location=point,
            radius=3,
            color='blue',
            fill=True,
            fill_color='blue',
            popup='Train'
        ).add_to(wolf_map)

    # Add test data points and draw lines between them
    test_points = [(row['location-lat'], row['location-long']) for idx, row in test_data.iterrows()]
    folium.PolyLine(test_points, color="green", weight=2.5, opacity=1).add_to(wolf_map)
    for point in test_points:
        folium.CircleMarker(
            location=point,
            radius=3,
            color='green',
            fill=True,
            fill_color='green',
            popup='Test'
        ).add_to(wolf_map)

    # Mark the first test observation with a green cross
    folium.Marker(
        location=test_points[0],
        icon=folium.DivIcon(html=html_green_cross),
        popup='First Test Observation'
    ).add_to(wolf_map)

    # Add prediction points and draw lines between them
    prediction_points = list(zip(test_preds_lat, test_preds_long))
    folium.PolyLine(prediction_points, color="red", weight=2.5, opacity=1).add_to(wolf_map)
    for idx, point in enumerate(prediction_points):
        folium.CircleMarker(
            location=point,
            radius=3,
            color='red',
            fill=True,
            fill_color='red',
            popup=f'Predicted: {test_data.index[idx]}'
        ).add_to(wolf_map)

    # Save the map
    wolf_map.save(f'../Visualisation/ARIMA/wolf_{wolf_id}_map.html')

# Iterate through each wolf and create a map
for wolf_id, info in results.items():
    test_preds_lat = info['test_predictions'][0]
    test_preds_long = info['test_predictions'][1]
    wolf_data = data[data['individual-id'] == wolf_id]
    create_wolf_map(wolf_data, test_preds_lat, test_preds_long, wolf_id)


#### ARIMA accuracy for each wolf

In [ ]:
import numpy as np
from math import radians, cos, sin, asin, sqrt

# Haversine function definition
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great-circle distance between two points 
    on the Earth (specified in decimal degrees).
    """
    # Convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # Haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371  # Radius of Earth in kilometers
    return c * r

# Initialize results dictionary outside of the loop
results = {}

# Loop over each wolf's data
for wolf_id, group in data.groupby('individual-id'):
    split_index = int(len(group) * 0.8)
    test_data = group.iloc[split_index:]

    # Calculate the Haversine distance for each prediction in the test set
    distances = [
        haversine(pred_lon, pred_lat, act_lon, act_lat)
        for pred_lon, pred_lat, act_lon, act_lat in zip(test_preds_long, test_preds_lat, test_data['location-long'], test_data['location-lat'])
    ]

    # Calculate mean, median, MAE, and RMSE for the Haversine distances
    mean_distance_error = np.mean(distances)
    median_distance_error = np.median(distances)
    
    # Store results including Haversine distances
    results[wolf_id] = {
        'test_accuracy': {
            'MAE_lat': test_mae_lat,  
            'RMSE_lat': test_rmse_lat,
            'MAE_long': test_mae_long,
            'RMSE_long': test_rmse_long
        },
        'haversine_accuracy': {
            'Mean_Haversine_Distance': mean_distance_error,
            'Median_Haversine_Distance': median_distance_error
        }
    }

# Print the results for each wolf
for wolf_id, info in results.items():
    print(f"Wolf ID: {wolf_id}")
    print('MAE_lat:', info['test_accuracy']['MAE_lat'])
    print('RMSE_lat:', info['test_accuracy']['RMSE_lat'])
    print('MAE_long:', info['test_accuracy']['MAE_long'])
    print('RMSE_long:', info['test_accuracy']['RMSE_long'])
    print('Mean_Haversine_Distance:', info['haversine_accuracy']['Mean_Haversine_Distance'])
    print('Median_Haversine_Distance:', info['haversine_accuracy']['Median_Haversine_Distance'])
    print("---------------")

#### ARIMA average accuracy

In [ ]:
# Initialize variables to accumulate the metrics
total_mae_lat = 0
total_rmse_lat = 0
total_mae_long = 0
total_rmse_long = 0
total_mean_haversine_distance = 0
total_median_haversine_distance = 0
num_wolves = len(results)

# Loop over each wolf's results
for wolf_id, info in results.items():
    # Accumulate individual metrics
    total_mae_lat += info['test_accuracy']['MAE_lat']
    total_rmse_lat += info['test_accuracy']['RMSE_lat']
    total_mae_long += info['test_accuracy']['MAE_long']
    total_rmse_long += info['test_accuracy']['RMSE_long']
    total_mean_haversine_distance += info['haversine_accuracy']['Mean_Haversine_Distance']
    total_median_haversine_distance += info['haversine_accuracy']['Median_Haversine_Distance']

# Calculate average metrics
average_mae_lat = total_mae_lat / num_wolves
average_rmse_lat = total_rmse_lat / num_wolves
average_mae_long = total_mae_long / num_wolves
average_rmse_long = total_rmse_long / num_wolves
average_mean_haversine_distance = total_mean_haversine_distance / num_wolves
average_median_haversine_distance = total_median_haversine_distance / num_wolves

# Print average metrics
print("Average Metrics for All Wolves:")
print(f"Average MAE Latitude: {average_mae_lat:.4f}")
print(f"Average RMSE Latitude: {average_rmse_lat:.4f}")
print(f"Average MAE Longitude: {average_mae_long:.4f}")
print(f"Average RMSE Longitude: {average_rmse_long:.4f}")
print(f"Average Mean Haversine Distance: {average_mean_haversine_distance:.2f} km")
print(f"Average Median Haversine Distance: {average_median_haversine_distance:.2f} km")

#### Saving the results 

In [ ]:
import pandas as pd

for wolf_id, group in data.groupby('individual-id'):

    # Convert results dictionary to a DataFrame for easier CSV saving
    results_df = pd.DataFrame([results[wolf_id]])

    # Save results to a CSV file
    results_csv_path = f'../Results/ARIMA results/results_{wolf_id}.csv'
    results_df.to_csv(results_csv_path, index=False)